In [ ]:
import pandas as pd

In [ ]:

from google.colab import files

# Загрузка файла с локального компьютера
uploaded = files.upload()
#"""

In [ ]:
data = pd.read_csv('kc_house_data.csv')
data.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
data = data.drop("id", axis=1)
data = data.drop("date", axis=1)

In [ ]:
def outliers(df,ft):
    q1 = df[ft].quantile(0.25)
    q3 = df[ft].quantile(0.75)
    iqr = q3 - q1

    lower_limit = q1 - iqr *1.5
    upper_limit = q3 + iqr *1.5

    ls = df.index[(df[ft]<lower_limit) | (df[ft]>upper_limit)]

    return ls

def remove(df,ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

In [ ]:
numeric_columns = data.drop(["waterfront","view","zipcode","yr_renovated","grade","condition","floors","bathrooms","bedrooms"],axis=1).select_dtypes(include=['float64', 'int64']).columns
numeric_data = data[numeric_columns]
index_list = []
for featsure in numeric_columns:
    index_list.extend(outliers(data,featsure))
df = remove(data,index_list)

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
drop_columns = ['sqft_lot', 'floors', 'condition', 'yr_built',
       'yr_renovated', 'zipcode', 'long', 'sqft_lot15']

data = data.drop(columns=drop_columns)

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(data.drop('price', axis=1), data["price"], test_size=0.3)

In [ ]:
lasso = Lasso()
lasso.fit(xTrain, yTrain)
lasso.score(xTrain, yTrain)
lassoPred = lasso.predict(xTest)
lassoMse = mean_squared_error(yTest, lassoPred)
lassoR2 = r2_score(yTest, lassoPred)
ridge = Ridge()
ridge.fit(xTrain, yTrain)
ridge.score(xTrain, yTrain)
ridgePred = ridge.predict(xTest)
ridgeMse = mean_squared_error(yTest, ridgePred)
ridgeR2 = r2_score(yTest, ridgePred)

In [ ]:
lassoMse, lassoR2, ridgeMse, ridgeR2

(49143840594.83893, 0.6501642584962262, 49139625572.36936, 0.650194263589301)

In [ ]:
lassoMse, lassoR2, ridgeMse, ridgeR2

(49114115348.82622, 0.6489989069551299, 49115135195.86985, 0.6489916184714726)

In [ ]:
lasso = Lasso()
lasso.fit(xTrain, yTrain)
lasso.score(xTrain, yTrain)

In [ ]:
lassoPred = lasso.predict(xTest)
lassoMse = mean_squared_error(yTest, lassoPred)
lassoMse

40433854297.25446

In [ ]:
lassoR2 = r2_score(yTest, lassoPred)
lassoR2

0.6981910610986395

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge()
ridge.fit(xTrain, yTrain)
ridge.score(xTrain, yTrain)

0.6998175573367837

In [ ]:
ridgePred = ridge.predict(xTest)
ridgeMse = mean_squared_error(yTest, ridgePred)
ridgeMse

40422869507.89023

In [ ]:
ridgeR2 = r2_score(yTest, ridgePred)
ridgeR2

0.6982730544598883

In [ ]:
models_cross = pd.DataFrame({
    'Model': ['Lasso','Ridge'],
    'Score': [lassoR2, ridgeR2],
    'MSE': [lassoMse, ridgeMse]})

models_cross

,Model,Score,MSE
0,Lasso,0.698191,4.043385e+10
1,Ridge,0.698273,4.042287e+10
